# <b>[파이썬 머신러닝 완벽 가이드]</b> <br>
# Chapter 08 텍스트 분석

## 8.5 감성 분석 <br>
### 감성 분석 소개 <br>
- 감성 분석 (Sentiment Analysis)은 문서의 주관적인 감성/의견/감정/기분 등을 파악하기 위한 방법으로 소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되고 있음
- 감성 분석은 문서 내 텍스트가 나타내는 여러 가지 주관적인 단어와 문맥을 기반으로 감성 (Sentiment) 수치를 계산하는 방법을 이용함
- 이러한 감성 지수는 긍정 감성 지수와 부정 감성 지수로 구성되며 이들 지수를 합산해 긍정 감성 또는 부정 감성을 결정함
<br><br>
- 지도학습은 학습 데이터와 타깃 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 다른 데이터의 감성 분석을 예측하는 방법으로 일반적인 텍스트 기반의 분류와 거의 동일함
- 비지도학습은 'Lexicon'이라는 일종의 감성 어휘 사전을 이용함. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 문서의 긍정적, 부정적 감성 여부를 판단함

### 지도학습 기반 감성 분석 실습 - IMDB 영화평

In [1]:
import pandas as pd

review_df = pd.read_csv('C:/Users/kimhj/Desktop/ESAA/OB/Data/labeledTrainData.tsv', header=0, sep='\t', quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


- id: 각 데이터의 id
- sentiment: 영화평 (review)의 Sentiment 결과 값(Target Label). 1은 긍정적 평가. 0은 부정적 평가를 의미
- review: 영화평의 텍스트

In [2]:
print(review_df['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [4]:
import re

#<br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

#파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

In [5]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

#스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization 수행
#LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))])

#Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probs)))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [8]:
#스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
#LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lr_clf', LogisticRegression(C=10))])

#Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행. predict_proba()는 roc 때문에 수행
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred),
                                                roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598


### 비지도학습 기반 감성 분석 소개 <br>
- 비지도 감성 분석은 Lexicon을 기반으로 하며, Lexicon은 일반적으로 어휘집을 의미하지만 여기서는 주로 감성만을 분석하기 위해 지원하는 감성 어휘 사전임
- 감성 사전은 긍정 (Positive) 감성 또는 부정 (Negative) 감성의 정도를 의미하는 수치를 가지고 있으며 이를 감성 지수 (Polarity score)라고 함
- 이 감성 지수는 단어의 위치나 주변 단어, 문맥, POS (Part of Speech) 등을 참고해 결정됨
- 이러한 감성 사전을 구현한 대표격은 NLTK 패키지로, Lexicon 모듈도 포함되어 있음
<br><br>
- NLP에서 제공하는 WordNet 모듈은 방대한 영어 어휘 사전으로, 단순한 어휘 사전이 아닌 시맨틱 분석을 제공하는 어휘 사전임 (시맨틱: '문맥상 의미')
<br><br>
- WordNet은 시맨틱 정보를 제공하기 위해 각각의 품사로 구성된 개별 단어를 Synset (Sets of cognitive synonyms)이라는 개념을 이용해 표현함
- Synset은 단수한 하나의 단어가 아니라 그 단어가 가지는 문맥, 시맨틱 정보를 제공하는 WordNet의 핵심 개념임
<br><br>
- NLTK의 예측 성능은 그리 좋지 않으므로 실제 업무의 적용은 NLTK 패키지가 아닌 다른 감성 사전을 적용하는 것이 일반적임
    - `SentiWordNet`: NLTK 패키지의 WordNet과 유사하게 감성 단어 전용의 WordNet과 유사하게 감성 단어 전용의 WordNet를 구현한 것으로, WordNet의 Synset 개념을 감성 분석에 적용한 것임. WordNet의 Synset별로 3가지 감성 점수(sentiment score)를 할당함. 긍정 감성 지수는 해당 단어가 감성적으로 얼마나 긍정적인가를, 부정 지수는 얼마나 감성적으로 부정적인가를 수치로 나타낸 것이고 객관성 지수는 긍정/부정 감성 지수와 완전히 반대되는 개념으로 단어가 감성과 관계없이 얼마나 객관적인지를 수치로 나타낸 것임. 문장별로 단어들의 긍정 감성 지수와 부정 감성 지수를 합산하여 최종 감성 지수를 계산하고 이에 기반해 감성이 긍정인지 부정인지를 결정함
    - `VADER`: 주로 소셜 미디어의 텍스트에 대한 감성 분석을 제공하기 위한 패키지로, 뛰어난 감성 분석 결과를 제공하며 비교적 빠른 수행 시간을 보장해 대용량 텍스트 데이터에 잘 사용됨
    - `Pattern`: 예측 성능 측면에서 가장 주목받는 패키지이지만 파이썬 3.X 버전에서 호환이 되지 않음

### SentiWordNet을 이용한 감성 분석 <br>
#### [WordNet Synset과 SentiWordNet SentiSynset 클래스의 이해]

In [9]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package nps_chat to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nps_chat.zip.
[nltk_data]    | Downloading package omw to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\opinion_lexicon.zip.
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package paradigms to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data

[nltk_data]    |   Unzipping corpora\verbnet3.zip.
[nltk_data]    | Downloading package webtext to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\webtext.zip.
[nltk_data]    | Downloading package wmt15_eval to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package word2vec_sample to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\word2vec_sample.zip.
[nltk_data]    | Downloading package wordnet to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downloading package wordnet2021 to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package wordnet31 to
[nltk_data]    |     C:\Users\kimhj\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package 

True

In [10]:
from nltk.corpus import wordnet as wn

term = 'present'

#'present'라는 단어로 wordnet의 sysnets 생성
synsets = wn.synsets(term)
print('synsets() 반환 type:', type(synsets))
print('synsets() 반환 값 개수:', len(synsets))
print('synsets() 반환 값:', synsets)

synsets() 반환 type: <class 'list'>
synsets() 반환 값 개수: 18
synsets() 반환 값: [Synset('present.n.01'), Synset('present.n.02'), Synset('present.n.03'), Synset('show.v.01'), Synset('present.v.02'), Synset('stage.v.01'), Synset('present.v.04'), Synset('present.v.05'), Synset('award.v.01'), Synset('give.v.08'), Synset('deliver.v.01'), Synset('introduce.v.01'), Synset('portray.v.04'), Synset('confront.v.03'), Synset('present.v.12'), Synset('salute.v.06'), Synset('present.a.01'), Synset('present.a.02')]


- `synsets()` 호출 시 반환되는 것은 여러 개의 Synset 객체를 가지는 리스트로, 총 18개의 서로 다른 semantic을 가지는 synset 객체가 반환되었음
- Sysnet('present.n.01')와 같이 Synset 객체의 파라미터 'present.n.01'은 POS 태그를 나타냄
- 'present.n.01'에서 present는 의미, n은 명사 품사, 01은 present가 명사로서 가지는 의미가 여러 가지 있어서 이를 구분하는 인덱스임

In [11]:
for synset in synsets:
    print('##### Synset name: ', synset.name(), '#####')
    print('Pos: ', synset.lexname()) #품사 (Part of Speech)
    print('Definition: ', synset.definition()) #정의
    print('Lemas: ', synset.lemma_names()) #부명제

##### Synset name:  present.n.01 #####
Pos:  noun.time
Definition:  the period of time that is happening now; any continuous stretch of time including the moment of speech
Lemas:  ['present', 'nowadays']
##### Synset name:  present.n.02 #####
Pos:  noun.possession
Definition:  something presented as a gift
Lemas:  ['present']
##### Synset name:  present.n.03 #####
Pos:  noun.communication
Definition:  a verb tense that expresses actions or states at the time of speaking
Lemas:  ['present', 'present_tense']
##### Synset name:  show.v.01 #####
Pos:  verb.perception
Definition:  give an exhibition of to an interested audience
Lemas:  ['show', 'demo', 'exhibit', 'present', 'demonstrate']
##### Synset name:  present.v.02 #####
Pos:  verb.communication
Definition:  bring forward and present to the mind
Lemas:  ['present', 'represent', 'lay_out']
##### Synset name:  stage.v.01 #####
Pos:  verb.creation
Definition:  perform (a play), especially on a stage
Lemas:  ['stage', 'present', 'represen

- Synset('present.n.01')과 Synset('present.n.02')는 명사지만 서로 다른 의미를 가지고 있음
<br><br>
- WordNet은 어떤 어휘와 다른 어휘 간의 관계를 유사도로 나타낼 수 있음
- synset 객체는 단어 간의 유사도를 나타내기 위해서 path_similarity() 메소드를 제공함

In [12]:
#synset 객체를 단어별로 생성함
tree = wn.synset('tree.n.01')
lion = wn.synset('lion.n.01')
tiger = wn.synset('tiger.n.02')
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')

entities = [tree, lion, tiger, cat, dog]
similarities = []
entity_names = [entity.name().split('.')[0] for entity in entities]

#단어별 synset을 반복하면서 다른 단어의 synset과 유사도를 측정함
for entity in entities:
    similarity = [round(entity.path_similarity(compared_entity),2)
                 for compared_entity in entities]
    similarities.append(similarity)

#개별 단어별 synset과 다른 단어의 synset과의 유사도를 DataFrame 형태로 저장함
similarity_df = pd.DataFrame(similarities, columns=entity_names, index=entity_names)
similarity_df

,tree,lion,tiger,cat,dog
tree,1.00,0.07,0.07,0.08,0.12
lion,0.07,1.00,0.33,0.25,0.17
tiger,0.07,0.33,1.00,0.25,0.17
cat,0.08,0.25,0.25,1.00,0.20
dog,0.12,0.17,0.17,0.20,1.00


In [13]:
import nltk
from nltk.corpus import sentiwordnet as swm

senti_synsets = list(swm.senti_synsets('slow'))
print('senti_synsets() 반환 type :', type(senti_synsets))
print('senti_synsets() 반환 값 개수:', len(senti_synsets))
print('senti_synsets() 반환 값:', senti_synsets)

senti_synsets() 반환 type : <class 'list'>
senti_synsets() 반환 값 개수: 11
senti_synsets() 반환 값: [SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


- SentiSynset 객체는 단어의 감성을 나타내는 감성 지수와 객관성(감성과 반대)을 나타내는 객관성 지수를 가지고 있음
- 감성 지수는 다시 긍정 감성 지수와 부정 감성 지수로 나뉨
- 어떤 단어가 전혀 감성적이지 않으면 객관성 지수는 1이 되고, 감성 지수는 모두 0이 됨

In [15]:
import nltk
from nltk.corpus import sentiwordnet as swn

father = swn.senti_synset('father.n.01')
print('father 긍정감성 지수: ', father.pos_score())
print('father 부정감성 지수: ', father.neg_score())
print('father 객관성 지수: ', father.obj_score())
print('\n')

fabulous = swn.senti_synset('fabulous.a.01')
print('fabulous 긍정감성 지수: ', fabulous.pos_score())
print('fabulous 부정감성 지수: ', fabulous.neg_score())
print('fabulous 객관성 지수: ', fabulous.obj_score())

father 긍정감성 지수:  0.0
father 부정감성 지수:  0.0
father 객관성 지수:  1.0


fabulous 긍정감성 지수:  0.875
fabulous 부정감성 지수:  0.125
fabulous 객관성 지수:  0.0


#### [SentiWordNet을 이용한 영화 감상평 감성 분석]
1. 문서(Document)를 문장(Sentence) 단위로 분해
2. 다시 문장을 단어(Word) 단위로 토큰화하고 품사 분해
3. 품사 태깅된 단어 기반으로 synset 객체와 senti_synset 객체를 생성
4. Senti_synset에서 긍정 감성/부정 감성 지수를 구하고 이를 모두 합산해 특정 임계치 값 이상일 때 긍정 감성으로, 그렇지 않을 때는 부정 감성으로 결정

In [16]:
from nltk.corpus import wordnet as wn

#간단한 NLTK PennTreebank Tag를 기반으로 WordNet 기반의 품사 Tag로 변환
def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

def swn_polarity(text):
    #감성 지수 초기화
    sentiment = 0.0
    tokens_count = 0
    
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    #분해된 문장별로 단어 토큰 -> 품사 태깅 후에 SentiSynset 생성 -> 감성 지수 합산
    for raw_sentence in raw_sentences:
        #NTLK 기반의 품사 태깅 문장 추출
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            #WordNet 기반 품사 태깅과 어근 추출
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            
            #어근을 추출한 단어와 WordNet 기반 품사 태깅을 입력해 Synset 객체를 생성
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            #sentiwordnet의 감성 단어 분석으로 감성 synset 추출
            #모든 단어에 대해 긍정 감성 지수는 +로 부정 감성 지수는 -로 합산해 감성 지수 계산
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            tokens_count += 1
        
        if not tokens_count:
            return 0
        
        #총 score가 0 이상일 경우 긍정 (Positive) 1, 그렇지 않을 경우 부정 (Negative) 0 반환
        if sentiment >= 0:
            return 1
        return 0

In [19]:
review_df['preds'] = review_df['review'].apply(lambda x: swn_polarity(x))
y_target = review_df['sentiment'].values
preds = review_df['preds'].values

In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.metrics import recall_score, f1_score, roc_auc_score
import numpy as np

print(confusion_matrix(y_target, preds))
print('정확도: ', np.round(accuracy_score(y_target, preds), 4))
print('정밀도: ', np.round(precision_score(y_target, preds), 4))
print('재현율: ', np.round(recall_score(y_target, preds), 4))

[[7668 4832]
 [3636 8864]]
정확도:  0.6613
정밀도:  0.6472
재현율:  0.7091


### VADER를 이용한 감성 분석 <br>
- VADER는 소셜 미디어의 감성 분석 용도로 만들어진 룰 기반의 Lexicon으로, SentimentIntensityAnalyzer 클래스를 이용해 쉽게 감성 분석을 제공함
- VADER는 NLTK 패키지의 서브 모듈로 제공될 수도 있고 단독 패키지로 제공될 수도 있음

In [23]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

senti_analyzer = SentimentIntensityAnalyzer()
senti_scores = senti_analyzer.polarity_scores(review_df['review'][0])
print(senti_scores)

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}


- VADER를 이용하면 먼저 SentimentIntensityAnalyzer 객체를 생성한 뒤에 문서별로 polarity_scores() 메소드를 호출해 감성 점수를 구한 뒤, 해당 문서의 감성 점수가 특정 임계값 이상이면 긍정, 그렇지 않으면 부정으로 판단함
- SentimentIntensityAnalyzer 객체의 polarity_scores() 메소드는 딕셔너리 형태의 감성 점수를 반환함
    - 'neg': 부정 감성 지수, 
    - 'neu': 중립적인 감성 지수, 
    - 'pos': 긍정 감성 지수
    - compound: neg, neu, pos score를 적절히 조합해 -1에서 1 사이의 감성 지수를 표현한 값
- compound score를 기반으로 부정 감성 또는 긍정 감성 여부를 결정하는데, 보통 0.1 이상이면 긍정 감성, 그 이사임녀 부정 감성으로 판단하나 상황에 따라 이 임계값을 적절히 조정해 예측 성능을 조절함

In [24]:
def vader_polarity(review, threshold=0.1):
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    #compound 값에 기반해 threshold 입력값보다 크면 1, 그렇지 않으면 0을 반환
    agg_score = scores['compound']
    final_sentiment = 1 if agg_score >= threshold else 0
    return final_sentiment

#apply lambda 식을 이용해 레코드별로 vader_polarity()를 수행하고 결과를 'vader_preds'에 저장
review_df['vader_preds'] = review_df['review'].apply(lambda x: vader_polarity(x, 0.1))
y_target = review_df['sentiment'].values
vader_preds = review_df['vader_preds'].values

print(confusion_matrix(y_target, vader_preds))
print('정확도: ', np.round(accuracy_score(y_target, vader_preds), 4))
print('정밀도: ', np.round(precision_score(y_target, vader_preds), 4))
print('재현율: ', np.round(recall_score(y_target, vader_preds), 4))

[[ 6730  5770]
 [ 1857 10643]]
정확도:  0.6949
정밀도:  0.6484
재현율:  0.8514
